**Loading libraries**

In [1]:
import numpy as np
import shutil
from PIL import Image
import os

**Creating directories**

In [2]:
base_dir = 'sample_data'
class1_dir = os.path.join(base_dir, 'class1')
class2_dir = os.path.join(base_dir, 'class2')
os.makedirs(class1_dir, exist_ok=True)
os.makedirs(class2_dir, exist_ok=True)

**Generate and save random images**

In [3]:
def generate_random_images(save_dir, num_images):
    for i in range(num_images):
        img = Image.fromarray(np.uint8(np.random.rand(224, 224, 3)*255))
        img.save(os.path.join(save_dir, f'image_{i}.jpg'))

num_images_per_class = 100
generate_random_images(class1_dir, num_images_per_class)
generate_random_images(class2_dir, num_images_per_class)
print(f'Sample images are generated at {base_dir} with {num_images_per_class} images per class.')

Sample images are generated at sample_data with 100 images per class.


**Load libraries for *pre-trained* models**

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.legacy import Adam

In [5]:
# Load VGG16 model pre-trained on imagenet
base_model = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(224, 224, 3))

# Freeze all layers initially
for layer in base_model.layers:
    layer.trainable=False

# Create new model and add base model and new layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

2025-05-21 22:51:36.685169: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-05-21 22:51:36.685196: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-21 22:51:36.685205: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-21 22:51:36.685244: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-21 22:51:36.685264: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


**Load and preprocess the data**

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('/Users/nareshdhami/Desktop/Visual Code/DeepLearning/sample_data',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='binary')

# Train the model with frozen layers
model.fit(train_generator, epochs=10)

Found 200 images belonging to 2 classes.
Epoch 1/10


2025-05-21 22:51:39.122944: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 3s 314ms/step - loss: 3.7136 - accuracy: 0.5100
Epoch 2/10
7/7 [==============================] - 2s 308ms/step - loss: 6.0111 - accuracy: 0.5000
Epoch 3/10
7/7 [==============================] - 2s 308ms/step - loss: 3.1872 - accuracy: 0.5150
Epoch 4/10
7/7 [==============================] - 2s 308ms/step - loss: 1.8039 - accuracy: 0.5200
Epoch 5/10
7/7 [==============================] - 2s 311ms/step - loss: 1.3614 - accuracy: 0.5500
Epoch 6/10
7/7 [==============================] - 2s 308ms/step - loss: 1.1290 - accuracy: 0.5400
Epoch 7/10
7/7 [==============================] - 2s 308ms/step - loss: 0.8815 - accuracy: 0.5700
Epoch 8/10
7/7 [==============================] - 2s 308ms/step - loss: 0.7583 - accuracy: 0.5850
Epoch 9/10
7/7 [==============================] - 2s 308ms/step - loss: 0.7749 - accuracy: 0.5700
Epoch 10/10
7/7 [==============================] - 2s 308ms/step - loss: 0.7931 - accuracy: 0.5550


**Lets gradually unfreeze the frozen layers**

In [7]:
for layer in base_model.layers[-4:]:    # Unfreeze last 4 layers
    layer.trainable = True

# Compile model again with lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train_generator, epochs=10)

Epoch 1/10
7/7 [==============================] - 4s 415ms/step - loss: 0.7608 - accuracy: 0.5400
Epoch 2/10
7/7 [==============================] - 3s 374ms/step - loss: 0.7520 - accuracy: 0.5500
Epoch 3/10
7/7 [==============================] - 3s 383ms/step - loss: 0.7498 - accuracy: 0.5600
Epoch 4/10
7/7 [==============================] - 3s 379ms/step - loss: 0.7462 - accuracy: 0.5500
Epoch 5/10
7/7 [==============================] - 3s 429ms/step - loss: 0.7493 - accuracy: 0.5600
Epoch 6/10
7/7 [==============================] - 3s 378ms/step - loss: 0.7481 - accuracy: 0.5400
Epoch 7/10
7/7 [==============================] - 3s 378ms/step - loss: 0.7466 - accuracy: 0.5450
Epoch 8/10
7/7 [==============================] - 3s 377ms/step - loss: 0.7460 - accuracy: 0.5500
Epoch 9/10
7/7 [==============================] - 3s 377ms/step - loss: 0.7480 - accuracy: 0.5400
Epoch 10/10
7/7 [==============================] - 3s 378ms/step - loss: 0.7461 - accuracy: 0.5450
